
# Chapter 3 -> Spark ETL with Azure (Blob | ADLS)

Task to do 
1. Install required spark libraries
2. Create connection with Azure Blob storage
3. Read data from blob and store into dataframe
4. Transform data
5. write data into parquet file 
6. write data into JSON file

Reference:
https://learn.microsoft.com/en-us/azure/open-datasets/dataset-catalog

In [1]:
# First Load all the required library and also Start Spark Session
# Load all the required library
from pyspark.sql import SparkSession

In [2]:
#Start Spark Session
spark = SparkSession.builder.appName("chapter3").getOrCreate()
sqlContext = SparkSession(spark)
#Dont Show warning only error
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/09 21:17:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1. Create connection with Azure Blob storage

In [4]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = "r"

In [5]:

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),blob_sas_token)
print('Remote blob path: ' + wasbs_path)

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


3. Read data from blob and store into dataframe

In [6]:
df = spark.read.parquet(wasbs_path)

In [7]:
df.printSchema()

root
 |-- vendorID: string (nullable = true)
 |-- tpepPickupDateTime: timestamp (nullable = true)
 |-- tpepDropoffDateTime: timestamp (nullable = true)
 |-- passengerCount: integer (nullable = true)
 |-- tripDistance: double (nullable = true)
 |-- puLocationId: string (nullable = true)
 |-- doLocationId: string (nullable = true)
 |-- startLon: double (nullable = true)
 |-- startLat: double (nullable = true)
 |-- endLon: double (nullable = true)
 |-- endLat: double (nullable = true)
 |-- rateCodeId: integer (nullable = true)
 |-- storeAndFwdFlag: string (nullable = true)
 |-- paymentType: string (nullable = true)
 |-- fareAmount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mtaTax: double (nullable = true)
 |-- improvementSurcharge: string (nullable = true)
 |-- tipAmount: double (nullable = true)
 |-- tollsAmount: double (nullable = true)
 |-- totalAmount: double (nullable = true)
 |-- puYear: integer (nullable = true)
 |-- puMonth: integer (nullable = true)



In [8]:
df.show(n=2)

+--------+-------------------+-------------------+--------------+------------+------------+------------+----------+---------+----------+---------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|vendorID| tpepPickupDateTime|tpepDropoffDateTime|passengerCount|tripDistance|puLocationId|doLocationId|  startLon| startLat|    endLon|   endLat|rateCodeId|storeAndFwdFlag|paymentType|fareAmount|extra|mtaTax|improvementSurcharge|tipAmount|tollsAmount|totalAmount|puYear|puMonth|
+--------+-------------------+-------------------+--------------+------------+------------+------------+----------+---------+----------+---------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|     CMT|2012-02-29 23:53:14|2012-03-01 00:00:43|             1|         2.1|        null|        null|-73.980494|40.730601|-73.983532|40.752311|         1|   

4. Transform data

In [9]:
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('tempSource')

Register the DataFrame as a SQL temporary view: source


In [10]:
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM tempSource LIMIT 10'))

Displaying top 10 rows: 


DataFrame[vendorID: string, tpepPickupDateTime: timestamp, tpepDropoffDateTime: timestamp, passengerCount: int, tripDistance: double, puLocationId: string, doLocationId: string, startLon: double, startLat: double, endLon: double, endLat: double, rateCodeId: int, storeAndFwdFlag: string, paymentType: string, fareAmount: double, extra: double, mtaTax: double, improvementSurcharge: string, tipAmount: double, tollsAmount: double, totalAmount: double, puYear: int, puMonth: int]

In [ ]:
newdf = spark.sql('SELECT * FROM tempSource LIMIT 10')

5. write data into parquet file 
6. write data into JSON file

In [ ]:
newdf.write.format("parquet").option("compression","snappy").save("parquetdata",mode='append')

In [ ]:
newdf.write.format("csv").option("header","true").save("csvdata",mode='append')

In [ ]:
newdf.show()